# Training with the BubbleML Dataset
This notebook shows an example of how to setup training with the BubbleML dataset.
It uses a downsampled version of PB Subcooled with fewer simulations.

For help with loading the data, check the `data_loading` notebook!

In [1]:
import h5py
import torch
from torch.utils.data import ConcatDataset, Dataset, DataLoader
from neuralop.models import FNO
import torch.nn.functional as F
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.signal import savgol_filter
import numpy as np

c:\Users\Harsh\.conda\envs\bubble-sciml\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DEVICE = 'cpu'

### Create the PyTorch Dataset
We create a dataset that reads from a single HDF5 file, corresponding to one simulation. In this example, we only use one timestep for input to predict the following timestep. Thus, the input has three channels: temperature, x-velocity, and the y-velocity. It just predicts temperatures for the next step.

In [3]:
TEMPERATURE = 'temperature'
VELX = 'velx'
VELY = 'vely'

class HDF5Dataset(Dataset):
    def __init__(self, filename):
        self.filename = filename
        self.data = h5py.File(self.filename, 'r')
        self.timesteps = self.data[TEMPERATURE][:].shape[0]
    
    def __len__(self):
        return self.timesteps - 1

    def _get_input(self, idx):
        r"""
        The input is the temperature, x-velocity, and y-velocity at time == idx
        """
        temp = torch.from_numpy(self.data[TEMPERATURE][idx])
        velx = torch.from_numpy(self.data[VELX][idx])
        vely = torch.from_numpy(self.data[VELY][idx])
        # returns a stack with shape [3 x Y x X]
        return torch.stack((temp, velx, vely), dim=0)

    def _get_label(self, idx):
        r"""
        The output is the temperature at time == idx
        """
        return torch.from_numpy(self.data[TEMPERATURE][idx]).unsqueeze(0)
    
    def __getitem__(self, idx):
        r"""
        As input, get temperature and velocities at time == idx.
        As the output label, get the temperature at time == idx + 1.
        """
        input = self._get_input(idx)
        label = self._get_label(idx+1)
        return input, label

### Create a ConcatDataset and Loaders
In order to combine multiple simulations into one larger train/validation set, we use PyTorch's `ConcatDataset`. This is as simple as it sounds, you pass in a list of separate datasets and it concatenates them into one larger dataset. From the user perspective, this acts like a typical dataset. The datalaoders can use this `ConcatDataset` in the normal way.

In [4]:
train_files = ['Twall-100.hdf5', 'Twall-106.hdf5']
val_files = ['Twall-103.hdf5']

train_dataset = ConcatDataset(HDF5Dataset(file) for file in train_files)
val_dataset = ConcatDataset(HDF5Dataset(file) for file in val_files)

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False)

print(f'Train batches: {len(train_dataloader)}')
print(f'Val batches: {len(val_dataloader)}')

Train batches: 100
Val batches: 50


### Creating a Model
We use `neuralop`s implementation of the Fourier Neural Operator (FNO). It has 3 input channels because we input the `temperature`, `velx`, and `vely` for one timestep. It outputs one channels for the temperature at the following timestep. We keep the lowest (16, 16) modes. As the dataset resolution has been reduced, we can't keep many more modes than this. As this is a simpler example, we use only 64 hidden channels and 4 layers.

In [5]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
model = FNO(in_channels=3,    # 3 channels for temp, velx, vely
            out_channels=1,   # 1 channel for temp
            n_modes=(16, 16), # keep the lowest fourier modes
            hidden_channels=64,
            n_layers=4)

print(f'FNO uses {count_parameters(model)} parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

FNO uses 4769601 parameters


### Training!
We train the model a short number of epochs and optimize using an MSE loss between the predicted temperature and the true temperature according to the simulation. We plot the average validation loss for each epoch. We see that the validation loss decreases.

In [6]:
EPOCHS = 15

val_losses = []

for epoch in range(EPOCHS):
    model.train()
    for iter, (input, label) in enumerate(train_dataloader):
        input = input.to(DEVICE).float()
        print(input.shape)
        label = label.to(DEVICE).float()
        pred = model(input)
        optimizer.zero_grad()
        loss = F.mse_loss(pred, label)
        loss.backward()
        optimizer.step()

    val_loss = []
    model.eval()
    for iter, (input, label) in enumerate(val_dataloader):
        input = input.to(DEVICE).float()
        label = label.to(DEVICE).float()
        pred = model(input)
        loss = F.mse_loss(pred, label)
        val_loss.append(loss.detach().item())
    val_losses.append(torch.mean(torch.tensor(val_loss)))
    
plt.plot(val_losses)
plt.ylabel('MSE Loss')
plt.xlabel('Epoch')

torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])
torch.Size([4, 3, 48, 48])


KeyboardInterrupt: 

### One-step error
We visualize the one-step error for a sample output. We plot the ground-truth temperature, predicted temperature, and the absolute error between the two. We see that the predicted version is a decent approximation of the ground-truth (with more data and higher-resolution, it would of course look better.)

In [ ]:
input, label = val_dataset[35]
input = input.to(DEVICE).float().unsqueeze(0)
label = label.to(DEVICE).float()
pred = model(input)

label = label.squeeze().cpu().numpy()
pred = pred.squeeze().detach().cpu().numpy()
abs_err = np.abs(label - pred)

fig, ax = plt.subplots(1, 3, figsize=(10, 5))

data = {
    'Ground Truth': label,
    'Predicted': pred,
    'Abs. Error': abs_err
}

for idx, (key, im) in enumerate(data.items()):
    im = ax[idx].imshow(np.flipud(im))
    fig.colorbar(im, ax=ax[idx], shrink=0.45)
    ax[idx].set_title(key)
    ax[idx].set_xticks([])
    ax[idx].set_yticks([])